In [ ]:
import os
import sys
import glob
import shutil
from ipywidgets import interact
from copy import deepcopy
import matplotlib.pyplot as plt
print('OK')

In [ ]:
sys.path.append('..')

In [ ]:
from preprocessing.BatchCt import BatchCt
from dataset.dataset.dsindex import FilesIndex

### Global index

In [ ]:
DIR_TEST = '/notebooks/data/MRT/nci/*'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [ ]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

In [ ]:
ind = FilesIndex(DIR_TEST, dirs=True)

In [ ]:
paths = glob.glob(DIR_TEST)

### Create batch by subindexing

In [ ]:
batch = BatchCt(ind.index[[1, 100, 300, 400, 600]])

In [ ]:
batch.__dict__

In [ ]:
dicty = dict(zip(ind.index, paths))

### Test actions

* check load

In [ ]:
batch.load(all_patients_paths=dicty, 
           btype='dicom')
batch_copy = deepcopy(batch)

* check attrs

In [ ]:
batch._data.shape

In [ ]:
batch._lower_bounds

In [ ]:
batch._upper_bounds

In [ ]:
batch.history

In [ ]:
batch.index

* check resize, segment, dump

In [ ]:
batch = batch.resize(num_slices_new=128, num_x_new=256, num_y_new=256,
                     num_threads=20).segment(erosion_radius=2, num_threads=20)

* check history

In [ ]:
batch.history

* check dump

In [ ]:
batch.dump(DIR_DUMP)

### Build batch from dump and compare

In [ ]:
batch_from_dump = BatchCt(ind.index[[1, 100, 300, 400, 600]])

In [ ]:
paths_dumped = glob.glob(DIR_DUMP+'/*/*')

In [ ]:
paths_dumped

In [ ]:
ind_dumped = FilesIndex(DIR_DUMP +'*', dirs=True)

In [ ]:
dicty_dumped = dict(zip(ind_dumped.index, paths_dumped))

In [ ]:
batch_from_dump.load(all_patients_paths=dicty_dumped,
                     btype='blosc')

In [ ]:
batch_from_dump._data.shape

In [ ]:
def plot_slices(height, num_pers, *batches):
    fig, axes = plt.subplots(1, len(batches), figsize = (10, len(batches)*8))
    com_args = dict(clim=(-1200, 300), cmap=(plt.cm.gray))
    
    for i in range(len(batches)):
        axes[i].imshow(batches[i].get_axial_slice(num_pers, height), **com_args)
    
    plt.show()

In [ ]:
interact(lambda height: plot_slices(height, 2, batch_copy, batch,
                                    batch_from_dump), height=(0.01, 0.99, 0.01))

### XIP

In [ ]:
xip = batch.make_XIP()

In [ ]:
interact(lambda height: plot_slices(height, 2, batch_from_dump, xip), height=(0.01, 0.99, 0.01))